In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Pair Programming: MongoDB exercises to build familiarity

### Objectives
- Start and stop clients
- Create databases and collections
- Add, update, and drop objects
- Load and manage twitter tweets in Mongo

**Start up mongo session, and import MongoClient from pymongo.  Pip install if necessary.  Use MongoClient() to create a new client object. **

In [92]:
# lets confirm that mongo server is running
!ps awx | grep mongo

20369 s035  S+     1:10.35 mongod
22934 s042  Ss+    0:00.01 /bin/sh -c ps awx | grep mongo
22936 s042  R+     0:00.00 grep mongo


In [3]:
from pymongo import MongoClient

In [4]:
client = MongoClient()

**Use client.TAB to learn about the properties you can call on client.  Can you get the host, port, address, database names of the databases currently stored there?**

In [5]:
client.HOST

'localhost'

In [6]:
client.PORT

27017

In [7]:
client.address

('localhost', 27017)

In [11]:
client.database_names()
client.new_one

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'new_one')

**Pick one of the databases and create an object representing that database.**

In [12]:
db = client.new_one

Show the collections that are in there.

In [66]:
db.collection_names()

[]

Create a new collection with a name of your choice.

In [68]:
db.create_collection("funky_test")

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'funky_test'), 'funky_test')

In [69]:
db.collection_names()

['funky_test']

Get one of the collections and create a new object representing it.

In [70]:
funky_test = db.get_collection('funky_test')

In [71]:
type(funky_test)

pymongo.collection.Collection

How many documents are in there?

In [72]:
funky_test.count()

0

**Insert a document of your creation into your collection.**

In [73]:
funky_test.insert_one({"username": "erb", "name": "erbsman", "email": "erbstermeistro@gmail.com"})

**Use InsertOne, DeleteOne, and ReplaceOne to insert, delete, and replace three documents of your own creation.  Put the jobs in a list, and submit it in bulk.**

In [20]:
funky_test.bulk_write?

In [74]:
from pymongo import InsertOne, DeleteOne, ReplaceOne

In [75]:
requests = [InsertOne({'y': 1}), DeleteOne({'x': 1}), ReplaceOne({'w': 1}, {'z': 1}, upsert=True)]

In [76]:
result = db.funky_test.bulk_write(requests)

In [77]:
print(result.inserted_count)
print(result.deleted_count)
# upsert ? : upsert	boolean	Optional. If set to true, creates a new document when no document matches the query criteria. T
# the default value is false, which does not insert a new document when no match is found.

1
0


In [78]:
funky_test.insert_one({'y':1})

In [79]:
for i in funky_test.find():
    print(i)

{'_id': ObjectId('5a8add40ecff66523d6004e2'), 'username': 'erb', 'name': 'erbsman', 'email': 'erbstermeistro@gmail.com'}
{'_id': ObjectId('5a8add47ecff66523d6004e3'), 'y': 1}
{'_id': ObjectId('5a8add479c4de161f8b68017'), 'z': 1}
{'_id': ObjectId('5a8add4decff66523d6004e4'), 'y': 1}


**How many documents are in the collection now?**

In [80]:
funky_test.count()

4

**Find one of them.**

In [81]:
funky_test.find_one({"y":1})
# user_erb = users.find_one({"username": "erb"})

{'_id': ObjectId('5a8add47ecff66523d6004e3'), 'y': 1}

In [82]:
for i in funky_test.find({'y':1}):
    print(i)

{'_id': ObjectId('5a8add47ecff66523d6004e3'), 'y': 1}
{'_id': ObjectId('5a8add4decff66523d6004e4'), 'y': 1}


**Print out all documents in your collection.**

In [83]:
for funky_doc in funky_test.find():
    print(funky_doc)

{'_id': ObjectId('5a8add40ecff66523d6004e2'), 'username': 'erb', 'name': 'erbsman', 'email': 'erbstermeistro@gmail.com'}
{'_id': ObjectId('5a8add47ecff66523d6004e3'), 'y': 1}
{'_id': ObjectId('5a8add479c4de161f8b68017'), 'z': 1}
{'_id': ObjectId('5a8add4decff66523d6004e4'), 'y': 1}


*Append the ID's of each document to an empty list.  Then show the list.*

In [84]:

doc_ids = []
for funky_doc in funky_test.find():
    doc_ids.append(funky_doc['_id'])

doc_ids

[ObjectId('5a8add40ecff66523d6004e2'),
 ObjectId('5a8add47ecff66523d6004e3'),
 ObjectId('5a8add479c4de161f8b68017'),
 ObjectId('5a8add4decff66523d6004e4')]

In [ ]:
funky_test.find({'y':1})

Finally, drop your new collection.

In [ ]:
db.drop_collection("funky_test")

Get some Twitter.  For access tokens, go to [Twitter's API site](https://apps.twitter.com) and load your consumer_key, consumer_secret, access_token, and access_token_secret into .twitter_config on your home directory.  See yesterday's Twittter notebook.

In [22]:
# pip install requests_oauthlib
import requests
from requests_oauthlib import OAuth1
import cnfg
from pprint import pprint
import json
import tweepy

In [26]:
db = client.funky_test

In [27]:
# Our collection: 
tweets = db.new_tweet

In [28]:
auth = tweepy.OAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
auth.set_access_token(config["access_token"],
                      config["access_token_secret"])

api=tweepy.API(auth)

In [29]:
tweets

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'funky_test'), 'new_tweet')

In [31]:
results=[]


for tweet in tweepy.Cursor(api.search,q="synchronized skating").items(10):
    results.append(tweet)

In [32]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert_one(data)  # insert into mongodb

In [33]:
tweets.find_one()

{'_id': ObjectId('5a8acd07ecff66523d6004d3'),
 'datetime': datetime.datetime(2018, 2, 19, 12, 59, 37),
 'tweet': b'@NBCOlympics @Lilawerner and synchronized skating (obvi) @AdrianVSST'}

In [35]:
for tweet in tweets.find():
    print (tweet['tweet'])

b'@NBCOlympics @Lilawerner and synchronized skating (obvi) @AdrianVSST'
b'The Mount Pearl-Paradise beginner Starlites synchronized skating team is sending love and best wishes to\xe2\x80\xa6 https://t.co/SpnoPgUg92'
b'RT @FOTLsynchro: Synchronized Skating!! \xe2\x9b\xb8\xe2\x9d\x84\xef\xb8\x8f\xe2\x9b\xb8 #whynotsynchro\nhttps://t.co/QGCR3ixbPa https://t.co/kOgLLvlVMf'
b'RT @kittkat626: SYNCHRONIZED SKATING https://t.co/8DL7XNZ5Lf'
b'RT @kittkat626: SYNCHRONIZED SKATING https://t.co/8DL7XNZ5Lf'
b'RT @kittkat626: SYNCHRONIZED SKATING https://t.co/8DL7XNZ5Lf'
b'RT @kittkat626: SYNCHRONIZED SKATING https://t.co/8DL7XNZ5Lf'
b"It's something my dad kind of instilled in me.\n\nLet every sport get their chance!\n\nObviously some like American foo\xe2\x80\xa6 https://t.co/3lvDEOhYig"
b'RT @kittkat626: SYNCHRONIZED SKATING https://t.co/8DL7XNZ5Lf'
b'RT @caramgabriel: synchronized figure skating https://t.co/NqmztGxxnK'


In [36]:
for tweet in tweets.find():
    print (tweet['datetime'])

2018-02-19 12:59:37
2018-02-19 12:36:27
2018-02-19 12:24:29
2018-02-19 11:41:28
2018-02-19 11:20:18
2018-02-19 11:09:18
2018-02-19 11:08:05
2018-02-19 08:24:24
2018-02-19 07:13:21
2018-02-19 05:23:16
